<a href="https://colab.research.google.com/github/GreeneryScenery/SegmentPlants/blob/main/scripts/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Settings

In [ ]:
# @title Settings
WRITE : bool = True # @param {type:'boolean'}
READ : bool = True # @param {type:'boolean'}
CONTINUE : bool = False # @param {type:'boolean'}
WRITE_IMAGE : bool = True # @param {type:'boolean'}
READ_IMAGE : bool = True  # @param {type:'boolean'}
SAVE : bool = True # @param {type:'boolean'}
PATH : str = '/content/SegmentPlants/inputs/8/aligned' # @param {type:'string'}
DATES_PATH : str = 'SegmentPlants/inputs/8/dates.csv' # @param {type:'string'}
READ_PATH : str = '' # @param {type:'string'}
CONTINUE_PATH : str = '' # @param {type:'string'}
CLASSES : list[str] = ['mango', 'romaine lettuce', 'tomato'] # @param {type:'raw'}
INDEX_TO_CLASS : dict[int, str] = {0: 'mango', 1: 'romaine', 2: 'tomato'} # @param {type:'raw'}
INDEX_TO_SUBCLASS : dict[int, str] = {0: 'growing', 1: 'harvest', 2: 'ripe', 3: 'unripe'} # @param {type:'raw'}

## Loading

In [ ]:
import os
import sys
import math
import numpy as np
import pandas as pd
from PIL import Image, ImageOps
import cv2
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torchvision.ops import box_convert
import shutil
from copy import deepcopy
from shapely import geometry
import json

In [ ]:
!pip install supervision==0.16.0

In [ ]:
import supervision as sv

In [ ]:
!pip install neuralforecast

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import Autoformer
from neuralforecast.losses.pytorch import MAE

In [ ]:
!git clone https://github.com/GreeneryScenery/SegmentPlants.git

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
colours = {
    'dark blue': '#2861ae',
    'light blue': '#add8f6',
    'purple': '#c68bdd',
    'pink': '#f9c7e2',
    'yellow': '#f6e9ad',
    'dark green': '#28ae8b'
}
colour_palette = sv.ColorPalette.from_hex(colours.values())
box_annotator = sv.BoxAnnotator(color = colour_palette, thickness = 2, text_scale = 0.6, text_thickness = 1)
mask_annotator = sv.MaskAnnotator(color = colour_palette, opacity = 0.65)

In [ ]:
if WRITE:
  '''
  Setup Grounding DINO
  '''

  from huggingface_hub import hf_hub_download

  !git clone https://github.com/open-mmlab/mmdetection.git

  !pip install -r mmdetection/requirements/multimodal.txt

  !pip install -U openmim
  !mim install mmengine mmdet mmcv

  from transformers import BertConfig, BertModel
  from transformers import AutoTokenizer

  config = BertConfig.from_pretrained('bert-base-uncased')
  model = BertModel.from_pretrained('bert-base-uncased', add_pooling_layer = False, config = config)
  tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

  config.save_pretrained('mmdetection/bert-base-uncased')
  model.save_pretrained('mmdetection/bert-base-uncased')
  tokenizer.save_pretrained('mmdetection/bert-base-uncased')

  shutil.move('SegmentPlants/inputs/data/detection/config/config.py', 'mmdetection/configs/grounding_dino/config.py')

  hf_hub_download(repo_id = 'GreeneryScenery/Chronocrop', filename = 'detection_model.pth', local_dir = 'SegmentPlants/models')

  '''
  Setup SAM
  '''

  !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

  sys.path.append('..')
  from segment_anything import sam_model_registry, SamPredictor

  sam_checkpoint = 'sam_vit_h_4b8939.pth'
  model_type = 'vit_h'

  sam = sam_model_registry[model_type](checkpoint = sam_checkpoint)
  sam.to(device = DEVICE)

  predictor = SamPredictor(sam)

  '''
  Setup Filter
  '''

  def in_box(current_box, compare_box, threshold = 0.85) -> bool:
    current_x_min, current_y_min, current_x_max, current_y_max = current_box
    compare_x_min, compare_y_min, compare_x_max, compare_y_max = compare_box

    overlap_x_min = max(current_x_min, compare_x_min)
    overlap_y_min = max(current_y_min, compare_y_min)
    overlap_x_max = min(current_x_max, compare_x_max)
    overlap_y_max = min(current_y_max, compare_y_max)

    overlap_width = max(0, overlap_x_max - overlap_x_min)
    overlap_height = max(0, overlap_y_max - overlap_y_min)

    overlap_area = overlap_width * overlap_height

    smaller_box_area = (compare_x_max - compare_x_min) * (compare_y_max - compare_y_min)

    if overlap_area / smaller_box_area >= threshold:
        return True
    else:
        return False

if READ:
  '''
  Setup DINOv2 Image Classification
  '''

  hf_hub_download(repo_id = 'GreeneryScenery/Chronocrop', filename = 'subclassification_model.pt', local_dir = 'SegmentPlants/models')

  from SegmentPlants.scripts.model import Classifier

  classification_model = Classifier(len(INDEX_TO_CLASS), len(INDEX_TO_SUBCLASS))
  classification_model.load_state_dict(torch.load('SegmentPlants/models/subclassification_model.pt'))
  classification_model.eval()
  classification_model.to(DEVICE)

  transform = transforms.Compose([
      transforms.RandomResizedCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  def preprocess(img):
      img = transform(img)
      img = img[None, :]
      return img

In [ ]:
def resize_image(image : np.ndarray, expected_size : int) -> np.ndarray:
    height = image.shape[0]
    width = image.shape[1]
    new_width = expected_size
    new_height = expected_size

    if width > height:
        ratio = new_width / width
        new_height = int(height * ratio)
    else:
        ratio = new_height / height
        new_width = int(width * ratio)

    new_dimensions = (new_width, new_height)
    if ratio < 1:
        interpolation = cv2.INTER_AREA
    else:
        interpolation = cv2.INTER_CUBIC
    new_image = cv2.resize(image, new_dimensions, interpolation = interpolation)
    return new_image

In [ ]:
def add_padding(image : Image.Image, expected_size : tuple[int, int]) -> tuple[Image.Image, tuple[int, int]]:
    '''
    Add padding around image while repositioning a coordinate.

    :param image: PIL Image.
    :param expected_size: Expected size of new image in pixels.
    :param roi: Region of interest (coordinate) to reposition.
    :returns: A tuple of the padded image and repositioned roi.
    '''

    delta_width : int = expected_size[0] - image.size[0]
    delta_height : int = expected_size[1] - image.size[1]
    padding_width : int = delta_width // 2
    padding_height : int = delta_height // 2
    padding : tuple[int, int, int, int] = (padding_width, padding_height, delta_width - padding_width, delta_height - padding_height)
    new_image : Image.Image = ImageOps.expand(image, padding)
    return new_image

## Reading Images

In [ ]:
image_names = []

for image_name in os.listdir(PATH):
  image_names.append(image_name)

image_names = [str(i) + '.jpg' for i in sorted([int(num.split('.')[0]) for num in image_names])]

In [ ]:
shutil.rmtree('processed', ignore_errors = True)

In [ ]:
os.makedirs('processed', exist_ok = True)

image_paths = []

for image_name in image_names:
  image = Image.open(os.path.join(PATH, image_name))
  image = Image.fromarray(resize_image(np.array(ImageOps.exif_transpose(image)), 1500))
  image_path = f'processed/{image_name.split('.')[0]}.png'
  image.save(image_path)
  image_paths.append(image_path)

In [ ]:
os.makedirs('outputs', exist_ok = True)

if WRITE and WRITE_IMAGE:
  os.makedirs('outputs/grounding_dino', exist_ok = True)
  os.makedirs('outputs/sam', exist_ok = True)
  os.makedirs('outputs/filter', exist_ok = True)

if READ and READ_IMAGE:
  os.makedirs('outputs/stage', exist_ok = True)
  os.makedirs('outputs/classes', exist_ok = True)
  os.makedirs('outputs/growth', exist_ok = True)

# Writing

In [ ]:
if WRITE:
  final_xyxy = []
  final_masks = []
  final_labels = []
  final_polygons = []

In [ ]:
if WRITE:
  for count, image_path in enumerate(image_paths):
    print(f'Writing {count + 1} out of {len(image_paths)}.')

    output_name = image_path.split('/')[-1].split('.')[0]

    image_source_bgr = cv2.imread(image_path)
    image_source = cv2.cvtColor(image_source_bgr, cv2.COLOR_BGR2RGB)

    '''
    Grounding DINO
    '''

    TEXT_PROMPT = ' . '.join(CLASSES)
    TEXT_PROMPT = f'"{TEXT_PROMPT}"'

    !python mmdetection/demo/image_demo.py $image_path mmdetection/configs/grounding_dino/config.py --weights SegmentPlants/models/detection_model.pth --texts $TEXT_PROMPT --device $DEVICE

    with open(f'outputs/preds/{output_name}.json') as json_file:
      data = json.load(json_file)

    labels = np.array(data['labels'])
    scores = np.array(data['scores'])
    xyxy_grounding_dino = np.array(data['bboxes'])

    shutil.rmtree('outputs/preds')
    shutil.rmtree('outputs/vis')

    labels = np.take(np.array(CLASSES), labels).astype(str)

    boolean = scores > 0.3

    if sum(boolean) == 0:
      if WRITE_IMAGE:
        original_image = Image.fromarray(image_source)
        original_image.save(f'outputs/grounding_dino/{output_name}.png')
        original_image.save(f'outputs/sam/{output_name}.png')
        original_image.save(f'outputs/filter/{output_name}.png')

      final_polygons.append(None)
      final_xyxy.append(None)
      final_masks.append(None)
      final_labels.append(None)
      continue

    scores = scores[boolean]
    labels = labels[boolean]
    xyxy_grounding_dino = xyxy_grounding_dino[boolean]

    detections_grounding_dino = sv.Detections(xyxy=xyxy_grounding_dino)

    labels_grounding_dino = [
        f'{label} {score:.2f}'
        for label, score
        in zip(labels, scores)
    ]

    if WRITE_IMAGE:
      annotated_grounding_dino = image_source_bgr.copy()
      annotated_grounding_dino = box_annotator.annotate(scene = annotated_grounding_dino, detections = detections_grounding_dino, labels = labels_grounding_dino)

      grounding_dino_image = Image.fromarray(cv2.cvtColor(annotated_grounding_dino, cv2.COLOR_BGR2RGB))
      grounding_dino_image.save(f'outputs/grounding_dino/{output_name}.png')

    '''
    SAM
    '''

    predictor.set_image(image_source)

    transformed_boxes = predictor.transform.apply_boxes_torch(torch.from_numpy(xyxy_grounding_dino), image_source.shape[:2])

    centres_x = (transformed_boxes[:, 0] + transformed_boxes[:, 2]) // 2
    centres_y = (transformed_boxes[:, 1] + transformed_boxes[:, 3]) // 2

    centres = torch.stack((centres_x, centres_y), dim = 1).to(torch.int)

    masks, iou_predictions, low_res_masks = predictor.predict_torch(
        point_coords = centres.unsqueeze(1).to(DEVICE),
        point_labels = torch.from_numpy(np.full(len(transformed_boxes), 1, dtype = int)).unsqueeze(1).to(DEVICE),
        boxes = transformed_boxes.to(DEVICE),
        multimask_output = False,
    )

    if DEVICE == 'cpu':
      detections_sam = sv.Detections(
          xyxy = sv.mask_to_xyxy(masks = masks.numpy()[:, 0, :, :]),
          mask = masks.numpy()[:, 0, :, :],
          confidence = iou_predictions.numpy()[:, 0],
          class_id = np.array(labels_grounding_dino)
      )
    else:
      detections_sam = sv.Detections(
          xyxy = sv.mask_to_xyxy(masks = masks.cpu().numpy()[:, 0, :, :]),
          mask = masks.cpu().numpy()[:, 0, :, :],
          confidence = iou_predictions.cpu().numpy()[:, 0],
          class_id = np.array(labels_grounding_dino)
      )

    detections_sam = detections_sam.with_nms(threshold = 0.5, class_agnostic = True)
    masks_sam = detections_sam.mask
    xyxy_sam = detections_sam.xyxy
    labels_sam = detections_sam.class_id
    class_id_sam = np.arange(len(labels_sam))
    np.random.shuffle(class_id_sam)

    detections_sam = sv.Detections(xyxy = xyxy_sam, mask = masks_sam, class_id = class_id_sam)

    if WRITE_IMAGE:
      annotated_sam = mask_annotator.annotate(scene = image_source_bgr.copy(), detections = detections_sam)
      combined_sam = box_annotator.annotate(scene = annotated_sam, detections = detections_sam, labels = labels_sam)
      Image.fromarray(cv2.cvtColor(combined_sam, cv2.COLOR_BGR2RGB)).save(f'outputs/sam/{output_name}.png')

    predictor.reset_image()

    '''
    Filter
    '''

    masks_filter = detections_sam.mask
    xyxy_filter = detections_sam.xyxy
    labels_filter = labels_sam.copy()
    polygons_filter = []

    filter = []

    for i in range(len(xyxy_filter)):
      for j in range(len(xyxy_filter)):
        if j != i:
          if in_box(xyxy_filter[i], xyxy_filter[j]):
            filter.append(i)
            break

    for i in sorted(filter, reverse = True):
      masks_filter = np.delete(masks_filter, i, axis = 0)
      xyxy_filter = np.delete(xyxy_filter, i, axis = 0)
      labels_filter = np.delete(labels_filter, i, axis = 0)

    class_id_filter = np.arange(len(labels_filter))
    np.random.shuffle(class_id_filter)

    detections_filter = sv.Detections(
        xyxy = xyxy_filter,
        mask = masks_filter,
        class_id = class_id_filter
    )

    if WRITE_IMAGE:
      annotated_filter = mask_annotator.annotate(scene = image_source_bgr.copy(), detections = detections_filter)
      combined_filter = box_annotator.annotate(scene = annotated_filter, detections = detections_filter, labels = labels_filter)
      Image.fromarray(cv2.cvtColor(combined_filter, cv2.COLOR_BGR2RGB)).save(f'outputs/filter/{output_name}.png')

    for mask in masks_filter:
      polygons = sv.mask_to_polygons(mask)
      for p in enumerate(polygons):
        polygons[p[0]] = p[1].tolist()
      polygons_filter.append(polygons[sorted([(c,len(l)) for c,l in enumerate(polygons)], key = lambda t: t[1])[-1][0]])
    final_polygons.append(polygons_filter)

    final_xyxy.append(xyxy_filter.tolist())
    final_masks.append(masks_filter)
    final_labels.append(labels_filter.tolist())

In [ ]:
write_outputs = {
    'xyxy': final_xyxy,
    'polygons': final_polygons,
    'labels': final_labels
}

In [ ]:
if WRITE and SAVE:
  with open('write_outputs.json', mode = 'w') as f:
      json.dump(write_outputs, f)

In [ ]:
if WRITE and WRITE_IMAGE and SAVE:
  !zip -r outputs.zip outputs

# Reading

In [ ]:
if READ:
  if WRITE:
    with open('write_outputs.json') as json_file:
      write_outputs = json.load(json_file)
  else:
    with open(READ_PATH) as json_file:
      write_outputs = json.load(json_file)

  read_xyxy = write_outputs['xyxy']
  read_polygons = write_outputs['polygons']
  read_labels = write_outputs['labels']

  if CONTINUE:
    with open(CONTINUE_PATH) as json_file:
      read_outputs = json.load(json_file)
    centres = read_outputs['centres']
    areas = read_outputs['areas']
    forecasts = read_outputs['forecasts']
  else:
    centres = dict()
    areas = dict()
    forecasts = dict()

In [ ]:
dates = pd.read_csv(DATES_PATH)

In [ ]:
if READ:
  for i, image_path in enumerate(image_paths):
    print(f'Reading {i + 1} out of {len(image_paths)}.')

    output_name = image_path.split('/')[-1].split('.')[0]

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for key in areas:
      areas[key].append(None)
      forecasts[key].append(None)

    if read_xyxy[i] is not None and read_polygons[i] is not None:
      labels_stage = []
      class_id_stage = []

      labels_classes = []
      class_id_classes = []
      custom_colour_lookup_classes = []

      xyxy_final = []
      masks_dinov2 = []
      masks_sam = []

      labels_growth = []
      class_id_growth = []

      for j in range(len(read_xyxy[i])):
        masked_image = image_rgb.copy()
        bbox = read_xyxy[i][j]
        masked_image = masked_image[bbox[1] : bbox[3], bbox[0] : bbox[2]]

        '''
        Subclassification
        '''

        img = preprocess(Image.fromarray(masked_image))
        img = img.to(DEVICE)
        with torch.no_grad():
            result = classification_model(img)
        c = result[0]
        s = result[1]
        s = s.detach()
        s = s.detach()
        c = c.cpu()
        s = s.cpu()
        c = c.numpy()
        s = s.numpy()
        c = INDEX_TO_CLASS[np.argmax(c)]
        s = INDEX_TO_SUBCLASS[np.argmax(s)]

        labels_stage.append(str((c, s)))
        if s == 'harvest' or s == 'ripe':
          class_id_stage.append(0)
        else:
          class_id_stage.append(2)

        xyxy_final.append(read_xyxy[i][j])
        masks_sam.append(sv.polygon_to_mask(np.array(read_polygons[i][j]), (image_rgb.shape[1], image_rgb.shape[0])))

        '''
        Classes
        '''

        xyxy = read_xyxy[i][j]
        centre = (xyxy[0] + xyxy[2]) // 2, (xyxy[1] + xyxy[3]) // 2

        min_key = None
        min_dist = None
        min_centre = None
        for key in centres:
          centre_compare = centres[key]
          if (dist := math.sqrt(math.pow(abs(centre_compare[0] - centre[0]), 2) + math.pow(abs(centre_compare[1] - centre[1]), 2))) < 100:
            if (min_key is not None and dist < min_dist) or (min_key is None):
              min_key = key
              min_dist = dist
              min_centre = centre

        if min_key is not None:
          centres[min_key] = min_centre
          if min_key not in areas.keys():
            areas[min_key] = [None] * (i + 1)
            forecasts[min_key] = [None] * (i + 1)
          areas[min_key][-1] = geometry.Polygon(read_polygons[i][j]).area
        else:
          new_key = len(centres)
          centres[new_key] = centre
          areas[new_key] = [None] * (i + 1)
          forecasts[new_key] = [None] * (i + 1)
          areas[new_key][-1] = geometry.Polygon(read_polygons[i][j]).area

        if min_key is not None:
          labels_classes.append(f'{min_key}, {c}')
          class_id_classes.append(min_key)
          custom_colour_lookup_classes.append(min_key % len(colours))
        else:
          labels_classes.append(f'{new_key}, {c}')
          class_id_classes.append(new_key)
          custom_colour_lookup_classes.append(new_key % len(colours))

      '''
      Growth
      '''

      for key in areas:
        if areas[key][-1] != None:
          if sum(np.array(areas[key]) != None) > 3:
            plant_areas = areas[key]

            Y_df = dates.loc[:len(plant_areas) - 2].copy()
            Y_df.loc[:, 'y'] = plant_areas[:-1]
            Y_df['ds'] = pd.to_datetime(Y_df['ds'])

            Y_df = Y_df.dropna()

            A_df = dates.loc[:len(plant_areas) - 1].copy()
            A_df.loc[:, 'y'] = plant_areas
            A_df['ds'] = pd.to_datetime(A_df['ds'])

            models_neural = [
                Autoformer(
                  h = 1,
                  input_size = 2,
                  hidden_size = 16,
                  conv_hidden_size = 32,
                  n_head = 2,
                  loss = MAE(),
                  scaler_type = 'robust',
                  learning_rate = 0.006,
                  max_steps = 100,
                  val_check_steps = 50)
            ]

            nf = NeuralForecast(models = models_neural, freq = 'D')
            nf.fit(df = Y_df)

            Y_hat_neural_df = nf.predict()

            Y_hat_neural_df = Y_hat_neural_df.reset_index()
            Y_hat_neural_df = pd.concat([Y_df.rename(columns = {'y': 'Autoformer'}), Y_hat_neural_df])

            forecasts[key][-1] = Y_hat_neural_df.iloc[-1, -1]

            growth = A_df.iloc[-1, -1] > Y_hat_neural_df.iloc[-1, -1]

            if growth:
              labels_growth.append('Growing well!')
              class_id_growth.append(0)
            else:
              labels_growth.append('Slow growth!')
              class_id_growth.append(2)
          else:
            labels_growth.append('Growing...')
            class_id_growth.append(1)

      if READ_IMAGE:
        if xyxy_final != []:
          xyxy_final = np.array(xyxy_final)
          masks_sam = np.array(masks_sam).astype(bool)
          labels_stage = np.array(labels_stage)
          class_id_stage = np.array(class_id_stage)

          labels_growth = np.array(labels_growth)
          class_id_growth = np.array(class_id_growth)

          for m in ['sam']:
            detections_stage = sv.Detections(
                xyxy = xyxy_final,
                mask = globals()[f'masks_{m}'],
                class_id = class_id_stage
            )

            annotated_stage = mask_annotator.annotate(scene = image.copy(), detections = detections_stage, custom_color_lookup = class_id_stage)
            combined_stage = box_annotator.annotate(scene = annotated_stage, detections = detections_stage, labels = labels_stage)
            Image.fromarray(cv2.cvtColor(combined_stage, cv2.COLOR_BGR2RGB)).save(f'outputs/stage/{output_name}.png')

            labels_classes = np.array(labels_classes)
            class_id_classes = np.array(class_id_classes)
            custom_colour_lookup_classes = np.array(custom_colour_lookup_classes)

            detections_classes = sv.Detections(
                xyxy = xyxy_final,
                mask = globals()[f'masks_{m}'],
                class_id = class_id_classes
            )

            annotated_classes = mask_annotator.annotate(scene = image.copy(), detections = detections_classes, custom_color_lookup = custom_colour_lookup_classes)
            combined_classes = box_annotator.annotate(scene = annotated_classes, detections = detections_classes, labels = labels_classes)
            Image.fromarray(cv2.cvtColor(combined_classes, cv2.COLOR_BGR2RGB)).save(f'outputs/classes/{output_name}.png')

            detections_growth = sv.Detections(
                xyxy = xyxy_final,
                mask = globals()[f'masks_{m}'],
                class_id = class_id_growth
            )

            annotated_growth = mask_annotator.annotate(scene = image.copy(), detections = detections_growth, custom_color_lookup = class_id_growth)
            combined_growth = box_annotator.annotate(scene = annotated_growth, detections = detections_growth, labels = labels_growth)
            Image.fromarray(cv2.cvtColor(combined_growth, cv2.COLOR_BGR2RGB)).save(f'outputs/growth/{output_name}.png')
        else:
          Image.fromarray(image_rgb).save(f'outputs/stage/{output_name}.png')
          Image.fromarray(image_rgb).save(f'outputs/classes/{output_name}.png')
          Image.fromarray(image_rgb).save(f'outputs/growth/{output_name}.png')
    else:
      if READ_IMAGE:
        Image.fromarray(image_rgb).save(f'outputs/stage/{output_name}.png')
        Image.fromarray(image_rgb).save(f'outputs/classes/{output_name}.png')
        Image.fromarray(image_rgb).save(f'outputs/growth/{output_name}.png')

In [ ]:
read_outputs = {
    'centres': centres,
    'areas': areas,
    'forecasts': forecasts
}

In [ ]:
with open('read_outputs.json', mode='w') as f:
    json.dump(read_outputs, f)

In [ ]:
if READ_IMAGE and SAVE:
  !zip -r outputs.zip outputs

# Figures

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio

In [ ]:
os.makedirs('figures', exist_ok = True)

In [ ]:
with open('read_outputs.json') as json_file:
  read_outputs = json.load(json_file)

In [ ]:
for key in read_outputs['forecasts']:
  areas = read_outputs['areas'][key]
  forecasts = read_outputs['forecasts'][key]

  Y_df = dates.loc[:len(areas) - 2].copy()
  Y_df.loc[:, 'y'] = areas[:-1]
  Y_df['ds'] = pd.to_datetime(Y_df['ds'])
  Y_df = Y_df.dropna()

  A_df = dates.loc[:len(forecasts) - 1].copy()
  A_df.loc[:, 'y'] = forecasts
  A_df['ds'] = pd.to_datetime(A_df['ds'])
  A_df = A_df.dropna()

  fig = go.Figure()

  fig.add_trace(go.Scatter(x = Y_df.ds, y = Y_df['y'], name = 'Actual', line = dict(color = 'black')))
  fig.add_trace(go.Scatter(x = A_df.ds, y = A_df['y'], name = 'Forecasts', line = dict(color = '#555555')))

  fig.update_layout(title = f'Actual vs Forecasted Surface Areas', xaxis_title = 'Date', yaxis_title = 'Surface Area', legend = dict(x = 1, y = 1))

  pio.write_html(fig, f'figures/{key}.html')

  fig.show()

In [ ]:
if READ_IMAGE and SAVE:
  !zip -r figures.zip figures